# Quantum Key Distribution

In quantum key distribution (QKD), the use of randomness extractors is required to perform an essential subroutine known as privacy amplification.

A general QKD protocol has the following setup. 

  * **Raw Key Exchange:** Raw key is exchanged between two parties. This is the only part of QKD that requires quantum resources.
  * **Key Sifting:** Certain rounds are selected which will be processed into shared secret key. 
  * **Parameter Estimation:** Statistical parameters about the key exchange are estimated.
  * **Error Correction:** Both parties raw keys are manipulated into a jointly shared raw key, which may be partially known by an adversary.
  * **Privacy Amplification:** The jointly shared raw key is processed into a shorter, uniform and private shared secret key.

Importantly, some of these steps assume the two parties have initial uniform (and sometimes private) shared randomness. 


## Experimental CV-QKD
Following the experimental results of the continuous variable protocol outlined in \cite{jain2022practical}, we give pseudocode replicating the
privacy amplification step of their experiment using our extractor library \texttt{extlib}. This example can be easily adapted to any explicit QKD protocol. 
In this protocol, the quantum information is encoded into the amplitude and phase quadratures of the optical field. 
Alice encodes random bits, which she generates with a quantum random number generator (QRNG), by modulating the optical signal field to obtain coherent states.
The QRNG used to generate random bits is based on \cite{gabriel2010generator}, which consists of homodyne measurements of the vacuum state. 
Bob decodes this information using coherent detection, facilitated by a so-called local oscillator.
For security against a quantum adversary, a quantum proof randomness extractor must be used for privacy amplification.

All security parameters are set to $10^{-10}$, except for correctness error after sifting, which is $10^{-12}$.
The protocol starts with $\mathrm{N}=10^9$ key generation rounds, which becomes $\mathrm{N_{sifted}}= 9.88 \times 10^8$ after sifting. 
After parameter estimation and error correction using multi edge type low-density parity check error correcting codes, they are left with
$\mathrm{N_{EC}} = 8.69 \times 10^8$. Since measurements are performed in 2 quadratures, 
this gives $n = 1.738 \times 10^9$ raw key bits for privacy amplification
on both Alice and Bob's side, which should be identical (up to the errors of prior protocol steps).
Based on the security parameters, a final secret key length of $41378264$ is calculated.

The experimental implementation uses the $\Toeplitz$ extractor for privacy amplification, with a raw key input length 
of $n = 1.738 \times 10^9$, an output of $m = 41378264$ and therefore, a required seed length of $d = 1.738 \times 10^9 + 41378263$.
This privacy amplification step can be performed easily using our randomness extractor library \texttt{extlib} as follows in Fig. \ref{fig:pa}. 


In [ ]:
import extlib
def privacy_amplification(seed_bits, raw_key_bits, n = 1.738 * 10**9, m = 41378264):
  """ Perform privacy amplification for Practical 
  CV-QKD with composable security.

  Parameters
  ----------
  raw_key_bits : list of bits from 
    the results of measurements in the QKD protocol, 
    after sifting, error correction and parameter 
    estimation. 
  seed_bits : list of bits generated 
    independently of the QKD protocol
    using a QRNG.
  n: integer, the number of raw key bits.
  m: integer, the number of output secret key bits.

  Returns
  ---------
  list of bits
    The extracted output, which is 
    the shared secret key.
  """
  assert len(seed_bits) == n + m - 1
  assert len(raw_key_bits) == n
  toeplitz = extlib.Toeplitz(n, m)
  return toeplitz.extract(seed_bits, raw_key_bits)

## Extensions

The experimental demonstration above admits some improvements with our extractor library:

### Extension 1: Toeplitz extraction

Since the QRNG generating the seed bits is device-dependent the quality of the seed relies on the correct characterization and perfect modelling of the device.
This opens the possibility that the seed bits may only be somewhat random.
In this case, a shared secret key can still be generated, but each user must use the Toeplitz two-source (as opposed to seeded) randomness extractor extension from \texttt{extlib} and adjust their output length accordingly. 
Below we show an example of this when considering the QRNG generated seed bits have a min-entropy rate of 0.99 (as opposed to 1 if it had been perfectly random). 
Let $\boldsymbol{x}, \boldsymbol{y}$ be the raw key bits and seed bits respectively, then the output length is adjusted to $m' = \frac{1}{3} (41378264 - 2 (k_1 - n) + 2)$, where $k_1 = 0.99 \times n$ is the seed min-entropy.

\begin{algorithm}[H]
  \caption{Toeplitz two-source extraction.}\label{alg:toeplitz-2-source}
\begin{algorithmic}[1]
  \Function{ToeplitzTwoSourcePrivacyAmp}{x,y}
  \State $n \gets 1738000000$ \Comment{input length}
  \State $m \gets 41378264$ \Comment{original output length}
  \State $k_1 \gets 0.99 \times n$ \Comment{calculate seed min-entropy} 
  \State $m' \gets \frac{1}{3} (41378264 - 2 (k_1 - n) + 2)$ \Comment{new output length, given by \eqref{eq:toeplits-q-2-params}}
  \State \textbf{return} \texttt{extlib.Toeplitz(n, m).extract($\bm{x}, \bm{y}$)}
  \EndFunction
\end{algorithmic}
\end{algorithm}

### Extension 2: Trevisan Extraction

One could use the Trevisan extractor to generate almost the same amount of shared secret key, whilst reducing the size of the QRNG seed needed. 
The cost of this reduction in seed size is computational complexity, so the privacy amplification algorithm would run much slower. 

\begin{algorithm}[H]
  \caption{Trevisan seeded extraction.}\label{alg:trevisan-ext}
\begin{algorithmic}[1]
  \Function{TrevisanPrivacyAmp}{x,y}
  \State $n \gets 1738000000$ \Comment{input length}
  \State $t \gets 2 \lceil \log_2(n) + 1 - 2\log_2(\epsilon) \rceil$ \Comment{seed size for one-bit extraction}
  \State $r = 2 \exp(1)$ \Comment{overlap for initial weak design}
  \State $l \gets \max( 1 , \Bigl\lceil \frac{\log(m-r) - \log(t-r)}{\log(r) - log(r-1)} \Bigr \rceil)$ 
  \Comment{number of iterations of the weak design}
  \State $d \gets (l+1)\times t^2$ \Comment{seed length}
  \State \textbf{return} \texttt{extlib.Trevisan(n, d, m).extract($\bm{x}, \bm{y}$)}
  \EndFunction
\end{algorithmic}
\end{algorithm}